## Track Stocks in Excel

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import yfinance as yf
import os
from pathlib import Path

In [ ]:
Stock_List = ['AAPL','NVDA','SA59.SI','S58.SI']
start_date = datetime(datetime.now().year - 1, datetime.now().month, datetime.now().day)
end_date = datetime.now()

project_folder = Path(__file__).parent / "data"

for stock in Stock_List:
    #fetch Data from Yahoo Finance
    if os.path.exists(project_folder / f"{stock}_data.csv"):
        df = pd.read_csv(project_folder / f"{stock}_data.csv", index_col=0, parse_dates=True)
        
    


In [3]:
from datetime import datetime
from pathlib import Path
import yfinance as yf
import pandas as pd

# --- Settings ---
stock = "AAPL"
start_date = datetime(datetime.now().year - 1, datetime.now().month, datetime.now().day)
end_date = datetime.now()

# --- Folder setup ---
data_folder = Path.cwd() / "data"
data_folder.mkdir(exist_ok=True)

# --- Download data ---
print(f"⬇️ Downloading {stock} data...")
df = yf.download(stock, start=start_date, end=end_date, auto_adjust=False)

# Handle multi-index columns
if isinstance(df.columns, pd.MultiIndex):
    df.columns = [col[-1] for col in df.columns.values]

# --- Save to CSV ---
file_path = data_folder / f"{stock}_data.csv"
df.to_csv(file_path)

print(f"💾 Data saved to: {file_path}")

# --- Show sample output ---
print(df.tail(5).round(2))


[*********************100%***********************]  1 of 1 completed

⬇️ Downloading AAPL data...
💾 Data saved to: c:\Users\shenx\OneDrive\Documents\GitHub\Stock-moitoring-dashboard\data\AAPL_data.csv
              AAPL    AAPL    AAPL    AAPL    AAPL      AAPL
Date                                                        
2025-10-06  256.69  256.69  259.07  255.05  257.99  44664100
2025-10-07  256.48  256.48  257.40  255.43  256.81  31955800
2025-10-08  258.06  258.06  258.52  256.11  256.52  36496900
2025-10-09  254.04  254.04  258.00  253.14  257.81  38322000
2025-10-10  245.27  245.27  256.38  244.00  254.94  61782400
